In [63]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:  # Score of 4 or 5
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    def evenly_distribute(self):
        negative = list(filter (lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter (lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        print(negative[0].text)
        print(len(negative))
        print(len(positive))
        

### Load Data

In [65]:
import json

file_name = './model/books_small_10000.json'


reviews = []  # empty list

with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text


'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

### Prep Data

In [67]:
from sklearn.model_selection import train_test_split

training,test = train_test_split(reviews, test_size =0.33, random_state=42)

cont = ReviewContainer(training)
cont.evenly_distribute()

It was just one of those books that never went anywhere. I like books that get your attention in the beginning and not drag out until a quarter way through. I decided to give it an early death - delete!
436
5611


In [68]:
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

train_x[0]
train_y[0]

'POSITIVE'

#### Bag of words vectorization

In [69]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())


Olivia Hampton arrives at the Dunraven family home as cataloger of their extensive library. What she doesn't expect is a broken carriage wheel on the way. Nor a young girl whose mind is clearly gone, an old man in need of care himself (and doesn&#8217;t quite seem all there in Olivia&#8217;s opinion). Furthermore, Marion Dunraven, the only sane one of the bunch and the one Olivia is inexplicable drawn to, seems captive to everyone in the dusty old house. More importantly, she doesn't expect to fall in love with Dunraven's daughter Marion.Can Olivia truly believe the stories of sadness and death that surround the house, or are they all just local neighborhood rumor?Was that carriage trouble just a coincidence or a supernatural sign to stay away? If she remains, will the Castle&#8217;s dark shadows take Olivia down with them or will she and Marion long enough to declare their love?Patty G. Henderson has created an atmospheric and intriguing story in her Gothic tale. I found this to be an

## Classification

#### Linear SVM

In [70]:
from sklearn import svm

clf_svm = svm.SVC(kernel = 'linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Decision Tree

In [71]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])


array(['POSITIVE'], dtype='<U8')

#### Naive Bayes

In [72]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = DecisionTreeClassifier()

clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(test_x_vectors[0])


array(['POSITIVE'], dtype='<U8')

#### Logistic Regression

In [73]:
from sklearn.linear_model import LogisticRegression

clf_lg = LogisticRegression()

clf_lg.fit(train_x_vectors, train_y)

clf_lg.predict(test_x_vectors[0])

/Users/praiseayoade/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['POSITIVE'], dtype='<U8')

## Evaluation

In [74]:
# Mean Accuracy 
print (clf_svm.score(test_x_vectors, test_y))
print (clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y)) 
print(clf_lg.score(test_x_vectors, test_y))

0.8124242424242424
0.7736363636363637
0.7672727272727272
0.8418181818181818


In [75]:
# F1 Scores
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average =None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
f1_score(test_y, clf_gnb.predict(test_x_vectors), average =None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])

array([0.8741769 , 0.15151515, 0.16020672])

In [76]:
test_set = ['very fun', "bad book do not buy", 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

## Turning Model with Grid search

In [77]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C' : (1,4,8,16,32)}

svc =svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [78]:
print (clf_svm.score(test_x_vectors, test_y))

0.8124242424242424


In [79]:
# good! good

## Saving Model

In [80]:
import pickle

with open ('./model/sentiment_analyis.pkl', 'wb') as f:
    pickle.dump(clf, f)

## Load model

In [81]:
with open ('./model/sentiment_analyis.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [82]:
print(test_x[1])

loaded_clf.predict(test_x_vectors[0])


Yet another wonderful book by Elle. I really hope there are more in this series. They're lol funny, have you in tears, and have your nerves going all in one. I normally don't write reviews (I always rare, which Elle is almost always a five star in my opinion), but as other reviewers said, this was the best out of the three. And there could be so much more to their stories....I want to know more!!! Keep up the wonderful writing Elle. I love your work.


array(['POSITIVE'], dtype='<U8')